In [1]:
import os
import zipfile
from dataclasses import dataclass
from pathlib import Path

from summarylm.logging import logger
from summarylm.constants import *
from summarylm.utils.common import read_yaml, create_directories, get_size

In [2]:
os.chdir("../")

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    ALL_HUGGINGFACE_DATA: list
    LOCAL_DATA_FILE: list

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            ALL_HUGGINGFACE_DATA=config.ALL_HUGGINGFACE_DATA,
            LOCAL_DATA_FILE = config.LOCAL_DATA_FILE,
        )

        return data_ingestion_config

In [14]:
from datasets import load_dataset

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        for i in range(len(self.config.LOCAL_DATA_FILE)):
            if not os.path.exists(self.config.LOCAL_DATA_FILE[i]):
                dataset = load_dataset(self.config.ALL_HUGGINGFACE_DATA[i])
                dataset.save_to_disk(self.config.LOCAL_DATA_FILE[i])
                logger.info(f"{self.config.ALL_HUGGINGFACE_DATA[i]} downloaded!")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.LOCAL_DATA_FILE[i]))}")

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e 

[2024-05-23 07:53:04,706: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2024-05-23 07:53:04,709: INFO: common: Yaml file: params.yaml loaded successfully]
[2024-05-23 07:53:04,710: INFO: common: Directory created successfully at: artifacts]
[2024-05-23 07:53:04,711: INFO: common: Directory created successfully at: artifacts/data_ingestion]
[2024-05-23 07:53:04,711: INFO: 368978256: File already exists of size: ~0 KB]


Saving the dataset (1/1 shards): 100%|██████████| 5622/5622 [00:00<00:00, 15216.24 examples/s]

[2024-05-23 07:54:25,968: INFO: 368978256: multi_news downloaded!]
